## Converting Saildrone data to IMMA1 format

#### Developed by Senya Stein (senyarocks11@gmail.com) in July 2019 for use with Saildrone data

#### IMMA documentation is at https://rda.ucar.edu/datasets/ds548.0/#!docs

#### IMMA files can be read using code developed by Phillip Brohan and modified by Zhankun Wang: https://github.com/oldweather/IMMA/blob/master/Python/IMMA/icoads.py

### Note: IMMA expects specific units and instrumentation, this program assumes the following incoming data, instruments, and country codes:

Time measurements- UTC 

Latitude- Degrees North (+ accuracy)

Longitude- Degrees East (+ accuracy)

Time- Hours plus minutes

Country that recruited ship- United States (C1_var, will need manual change)

Wind direction compass precision- Tenths of degrees (+ accuracy)

Wind speed units- meters per second, obtained from anemometer (measured) at height of 4.5 meters

Sea level pressure- Hpa

Sea surface temperature- taken with a through hull sensor

Wave period- seconds

Wave height- Meters

COG: degrees

SOG: meters per second


Variable names correspond to those listed in the IMMA format document release 3.0 (linked above)


In [58]:
import numpy as np
import pandas as pn
import xarray as xr
import matplotlib.pyplot as plt
import datetime as dt
import netCDF4

In [59]:
###change url variable, input your data

#url = 'https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/allData/insitu/L2/saildrone/Baja/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
url = './../../data/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'

ds = xr.open_dataset(url, drop_variables = {'WING_ANGLE','BARO_PRES_STDDEV', 'ROLL', 'PITCH', 'TEMP_AIR_STDDEV', 'RH_STDDEV', 'UWND_STDDEV', 'VWND_STDDEV', 'GUST_WND_STDDEV', 'TEMP_CTD_STDDEV', 'COND_STDDEV', 'SAL_STDDEV', 'O2_CONC_UNCOR_MEAN', 'O2_CONC_UNCOR_STDDEV', 'O2_SAT_MEAN', 'O2_SAT_STDDEV', 'TEMP_O2_MEAN', 'TEMP_O2_STDDEV', 'CHLOR_MEAN', 'CHLOR_STDDEV', 'BKSCT_RED_MEAN', 'BKSCT_RED_STDDEV', 'CDOM_STDDEV', ' WWND_STDDEV', 'TEMP_IR_UNCOR_STDDEV' })
ds

<xarray.Dataset>
Dimensions:             (obs: 86839, trajectory: 1)
Coordinates:
  * trajectory          (trajectory) float32 1002.0
    time                (trajectory, obs) datetime64[ns] ...
    latitude            (trajectory, obs) float64 ...
    longitude           (trajectory, obs) float64 ...
Dimensions without coordinates: obs
Data variables:
    SOG                 (trajectory, obs) float64 ...
    COG                 (trajectory, obs) float64 ...
    HDG                 (trajectory, obs) float64 ...
    HDG_WING            (trajectory, obs) float64 ...
    BARO_PRES_MEAN      (trajectory, obs) float64 ...
    TEMP_AIR_MEAN       (trajectory, obs) float64 ...
    RH_MEAN             (trajectory, obs) float64 ...
    TEMP_IR_UNCOR_MEAN  (trajectory, obs) float64 ...
    UWND_MEAN           (trajectory, obs) float64 ...
    VWND_MEAN           (trajectory, obs) float64 ...
    WWND_MEAN           (trajectory, obs) float64 ...
    WWND_STDDEV         (trajectory, obs) float64 .

In [60]:
#after running this block once, you wont have to run it again (per data set) because resampled data will be downloaded 
# swap obs for time
ds = ds.isel(trajectory=0)
ds = ds.swap_dims({'obs':'time'})

#resample data to increments of 1 hr
dshr = ds.resample(time='1h', skipna=True, label='left').mean()
dshr.to_netcdf('./../../data/saildrone_resampled.nc')


KeyboardInterrupt: 

In [61]:
size = int(len(dshr['time']))

In [63]:
dshr = xr.open_dataset('./../../data/saildrone_resampled.nc')
#define variables 
#'   ' corresponds to when incoming saildrone data doesnt include this variable
time_shift = dshr.time + np.timedelta64(30,'m')
lat_shift = ds.latitude
lon_shift = ds.longitude
IM = '01' #IMMA version
ATTC = ' ' #attm count
TI_var = '2' #time indicator
LI_var = '5' #latitude indicator
DSDV = '  ' #ship course and ship speed (within past three hours)
NID = '  ' #national source indicator
II = '  ' #ID indicator
ID = '         ' #callsign
C1 = '02' ### C1 is The country that recruited a ship, which may differ from the country of immediate receipt, see page 21, # 16 in IMMA documentation for full list of codes
VItoW1 = '      ' #VV indicator through past weather
AtoPPP = '  ' #characteristic of PPP through amt pressure tend
IT_var = '8' #indicator for temperatures
WBTItoDPT = '          ' #WBT indicator through dew-point temperature

SI_var = '04' #SST measure method
NtoCH = '       ' #total cloud amt though high cloud type
SDtoHDG = '' #swell direction through ship's heading (154 chars)
ALL_var = '' #the rest of the unused variables following SOG (458 chars)
for i in range(154-1):
    SDtoHDG += ' '
for i in range(458-1):
       ALL_var += ' '
        
for i in range(size-1):
    #format string output and put hours in correct units (.01hr)
    HR = float(time_shift.dt.hour[i]+time_shift.dt.minute[i]/60)
    HR = "{0:.2f}".format(HR).zfill(5)
    HR = str(HR)
           
    time_str = str(time_shift.dt.year[i].data)+str(time_shift.dt.month[i].data).zfill(2)+str(time_shift.dt.day[i].data).zfill(2)+HR
    #time_str is YR + MO + DY + HR
    
    #format and put lat and lon in correct units
    LAT = float(lat_shift[i])
    LAT = "{0:.2f}".format(LAT).zfill(6)
    LON = float(lon_shift[i])
    LON = "{0:.2f}".format(LON).zfill(7)
    pos_str = str(LAT)+str(LON)
    
    DI_var = '6' #wind direction indicator
    WI_var = '4' #wind speed indicator
    
    if np.isnan(ds.UWND_MEAN[i]) | np.isnan(ds.VWND_MEAN[i]):
        D_var = '   ' #wind direction
    else:
        #convert U and V vectors to direction
        D_var = float(np.arctan2(ds.VWND_MEAN[i].data, ds.UWND_MEAN[i].data)*180/np.pi) 
        D_var = "{0:.0f}".format(D_var).zfill(3)
        D_var = str(D_var)
        
        WS_height=int(ds.UWND_MEAN.installed_height) #wind speed height
        #saildrone measures at 4.5 m, buoys are at 5m. Following changes speed to account for a 10m height
        UWND_MEAN = (ds.UWND_MEAN.data[i]*np.log(10./1e-4))/np.log(WS_height/1e-4)
        VWND_MEAN = (ds.VWND_MEAN.data[i]*np.log(10./1e-4))/np.log(WS_height/1e-4)
    
        #W-var is wind speed
        W_var = np.sqrt(ds.UWND_MEAN[i].data**2 + ds.VWND_MEAN[i].data**2) #convert vectors to speed
        W_var = float(W_var/10) #unit conversion (1 m/s to .1 m/s)
        W_var = "{0:.1f}".format(W_var).zfill(4) 
        W_var = str(W_var)
    
    #reformat the SLP and put in correct units
    if np.isnan(ds.BARO_PRES_MEAN[i]):
        SLP_var = '      ' #sea level pressure
    else:
        SLP_var = float(ds.BARO_PRES_MEAN[i].data*.1)
        SLP_var = "{0:.1f}".format(SLP_var).zfill(6)
        SLP_var = str(SLP_var)
    
    if np.isnan(ds.TEMP_AIR_MEAN[i]):
        AT_var = '     ' #air temperature
    else:
        AT_var = ds.TEMP_AIR_MEAN[i]
        AT_var = float(AT_var.data)*.1
        AT_var = "{0:.1f}".format(AT_var).zfill(5)
        AT_var = str(AT_var)
        
    if np.isnan(ds.TEMP_CTD_MEAN[i]):
        SST_var = '     ' #sea surface temperature
    else:
        SST_var = ds.TEMP_CTD_MEAN[i]
        SST_var = float(SST_var.data)*.1
        SST_var = "{0:.1f}".format(SST_var).zfill(5)
        SST_var = str(SST_var)
        
    if 'WAVE_DOMINANT_PERIOD' in locals(): #if this variable exists
        #wave period
        WD_var = '  '
        WP_var = str(WAVE_DOMINANT_PERIOD[i].data)
        WH_var = float((WAVE_SIGNIFICANT_HEIGHT[i].data)/2)
        WH_var = str(WH_var)
    else:
        WD_var = '  ' #wave direction
        WP_var = '  ' #wave period
        WH_var = '  '#wave height
        
    COG_var = ds.COG #course over ground
    SOG_var = ds.SOG #speed over ground
    
    COG_var = float(COG_var[i].data)
    COG_var = "{0:.0f}".format(COG_var).zfill(3)
    COG_var = str(COG_var)
    
    SOG_var = float((SOG_var[i].data)*1.944)
    SOG_var = "{0:.0f}".format(SOG_var).zfill(2)
    SOG_var = str(SOG_var)
        
    #combine all variables into one big string 
    Final_IMMA1_str = time_str + pos_str + IM + ATTC + TI_var + LI_var + DSDV + NID + II + ID + C1 + DI_var + D_var + WI_var + W_var + VItoW1 + SLP_var + AtoPPP + IT_var + AT_var + WBTItoDPT + SI_var + SST_var + NtoCH + WD_var + WP_var + WH_var + SDtoHDG + COG_var + SOG_var + ALL_var + '\n' 
    
    ###name or create your file to write IMMA data to:
    f = open("./../../Data/IMMA_Data/IMMA_saildrone_test.imma", 'a+')
    
    f.write(Final_IMMA1_str)
    f.close
    